In [1]:
import os, shutil
from cogent import LoadTable

In [2]:
def format_direction(from_to):
    return r"%s$\rightarrow$%s" % tuple(from_to.split("to"))

def format_positions(positions):
    positions = positions.split(":")
    old_new = {"0": "-2", "1": "-1", "2": "+1", "3": "+2"}
    redone = [old_new[p[-1]] for p in positions]
    if len(redone) == 1:
        template = "%s"
    else:
        template = "(%s)"
    
    return template % ", ".join(redone)

def latex_format_numbers(value):
    try:
        value = float(value)
    except ValueError:
        return value
    
    if value == 0:
        return "0"
    v = "%.1e" % value
    v = v.split("e")
    v[1] = str(int(v[1]))
    v = r"$%s\times 10^{%s}$" % tuple(v)
    return v

def format_pvalue(value):
    """0.0 if 0, float if <= 0.0001, scientific otherwise"""
    epsilon = 1e-6
    if value <= 1e-100:
        result = "0.0"
    elif 1-epsilon <= value <= 1.0+epsilon:
        result = "1.0"
    elif 1e-4 <= value:
        result = "%.4f" % value
    elif 0 < value < 1e-4:
        result = latex_format_numbers(value)
    else:
        raise ValueError(value)
    return result

table_template = "\\begin{table}[htp]\n\\centering\n%s\n\end{table}\n"

def format_latex_table(table, justify, label=None):
    caption = table.Title or ""
    label = label or ""
    table.Title = ""
    result = table_template % table.tostring(format="latex", justify=justify)
    result = result.replace("longtable", "tabular").replace("[htp!]", "")
    
    if caption:
        caption = r"\caption{%s}" % caption
    
    if label:
        caption += r"\label{%s}" % label
    
    if caption:
        result = result.splitlines()
        result.insert(-1, caption)
        result = "\n".join(result)
    
    return result

In [3]:
RANK = 0
def SequentialSidak(N, alpha=0.05):
    # see https://www.utdallas.edu/~herve/abdi-Holm2010-pretty.pdf
    global RANK
    RANK = 0
    def call(val):
        global RANK
        corr = N - RANK
        adjusted_p = 1 - (1 - val)**corr
        RANK += 1
        return adjusted_p <= alpha
    return call

COLUMNS = ["Position", "RE", "Deviance", "df", "prob"]
def make_summary_table(fns, filter_func=None, show_significant=False):
    """filter_func takes a table and returns the table filtered for rows"""
    records = []
    for fn in fns:
        direction = fn.split("/")[-2]
        row = [format_direction(direction)]
        table = LoadTable(fn, sep="\t")
        table = table.getColumns(COLUMNS)
        table = table.withNewColumn("motif_length", lambda x: x.count(":") + 1, columns="Position")
        if filter_func is not None:
            table = filter_func(table)
        
        table = table.sorted("prob")
        is_sig = SequentialSidak(table.Shape[0])

        table = table.filtered(is_sig, columns="prob")
        if show_significant and table.Shape[0] > 0:
            print direction
            print table
            print
        
        for length in range(1, 5):
            subtable = table.filtered(lambda x: x == length, columns="motif_length")
            if subtable.Shape[0] == 0:
                row.extend(["-", "-"])
                continue

            pos, re = subtable.sorted("RE").getRawData(["Position", "RE"])[-1]
            pos = format_positions(pos)
            re = format_pvalue(re)
            row.append(re)
            row.append("%s" % pos)
        if set(row[1:]) == set(["-"]): # no effects
            continue
        records.append(row)
    
    if not records:
        return None
    header = ["Direction", r"RE$_{max}(1)$", "Pos.$(1)$", r"RE$_{max}(2)$", "Pos.$(2)$",
              r"RE$_{max}(3)$", "Pos.$(3)$", r"RE$_{max}(4)$", "Pos.$(4)$"]
    summary = LoadTable(header=header, rows=records)
    # check we got values for each column
    columns = []
    for c in summary.Header:
        values = summary.getDistinctValues(c)
        if values != set("-"):
            columns.append(c)
    
    return summary.getColumns(columns)

# Manuscript figures and tables

## Figures

In [4]:
!mkdir -p ../figures

In [5]:
!ls ../results/ensembl_snps_79/A/intergenic/directions/CtoT

1.json       2.pdf        4.json       MI.pdf       summary.txt
1.pdf        3.json       4.pdf        analysis.log
2.json       3.pdf        MI.log       summary.pdf


### C->T figure

Copy the 1,2,3,4 plots and the summary and place them into ../figures/

In [6]:
fns = !ls ../results/ensembl_snps_79/A/intergenic/directions/CtoT/*.pdf

In [7]:
for fn in fns:
    bn = os.path.basename(fn)
    bn = "A_intergenic_freq_All-chrom_A-GC_All-CtoT_%s" % bn
    ofn = os.path.join("../figures/", bn)
    shutil.copy(fn, ofn)

### A->G figure

In [8]:
fns = !ls ../results/ensembl_snps_79/A/intergenic/directions/AtoG/*.pdf
for fn in fns:
    bn = os.path.basename(fn)
    bn = "A_intergenic_freq_All-chrom_A-GC_All-AtoG_%s" % bn
    ofn = os.path.join("../figures/", bn)
    shutil.copy(fn, ofn)

### Melanoma grid figure

In [9]:
fn = "../results/ensembl_snps_79/malignant_melanoma/directions/nbr_grid.pdf"
ofn = os.path.join("../figures/", "malignant_melanoma_grid.pdf")
shutil.copy(fn, ofn)

### Spectra grid for intergenic Autosomal vs X-chromosomal

In [10]:
fns = !ls ../results/ensembl_snps_79/A_vs_X/intergenic/spectra_grid.pdf
assert len(fns) == 1, fns
fn = fns[0]
bn = os.path.basename(fn)
ofn = os.path.join("../figures/", "spectra_grid_A_vs_X.pdf")
shutil.copy(fn, ofn)

### Spectra grid got malignant melanoma strand

In [11]:
fns = !ls ../results/ensembl_snps_79/malignant_melanoma/combined_strand_symmetric/spectra_grid.pdf

assert len(fns) == 1, fns
fn = fns[0]
bn = os.path.basename(fn)
ofn = os.path.join("../figures/", "spectra_grid_melanoma_strand_symmetry.pdf")
shutil.copy(fn, ofn)

## Tables

In [12]:
all_ms_tables = []

### Make C->T table

In [13]:
fns = !ls ../results/ensembl_snps_79/A/intergenic/directions/CtoT/*.txt
table = LoadTable(fns[0], sep="\t")
table = table.sorted(columns=["df", "Position"])
table = table.withNewColumn("Position(s)", format_positions, columns=["Position"])
table = table.getColumns(["Position(s)", "Deviance", "df", "prob"])
table = table.withNewHeader("prob", "p-value")
table.setColumnFormat("p-value", format_pvalue)
table.setColumnFormat("Deviance", "%.1f")
table.Title = r"Log-linear analysis of C$\rightarrow$T autosomal intergenic mutations. "\
+r"Position(s) are relative to the index position (see Figure \ref{fig:nbr_indexing}). "\
+r"Deviance is from the log-linear model, with df degrees-of-freedom and corresponding "\
+r"p-value obtained from the $\chi^2$ distribution. p-values listed as 0.0 are below "\
+r"the limit of detection."
all_ms_tables.append(format_latex_table(table, justify="rrrl", label="table:c-t"))

### Intergenic autosomal mutation summary table

In [14]:
from mutation_motif.complement import MUTATION_COMPLEMENTS

fns = !ls ../results/ensembl_snps_79/A/intergenic/directions/*/*.txt
fns = [fn for fn in fns if fn.split("/")[-2] in MUTATION_COMPLEMENTS]

summary = make_summary_table(fns)
summary = summary.getColumns([c for c in summary.Header if "4" not in c])
summary.Title = "Summary of neighbourhood contributions to plus strand mutations "\
+r"with an autosomal intergenic location. RE$_{max}(\#)$ is the maximum RE for order "\
+r"\# and Pos.$(\#)$ the corresponding position(s). All point mutations had at least "\
+r"one significant test after correcting for 15 tests (see Table \ref{table:c-t}) using the "\
+r"Holm-{\v S}id{\"a}k procedure."
all_ms_tables.append(format_latex_table(summary, justify="crrrrrr", label="table:intergenic:summary"))

### Make A->G table

In [15]:
fns = !ls ../results/ensembl_snps_79/A/intergenic/directions/AtoG/*.txt
table = LoadTable(fns[0], sep="\t")
table = table.sorted(columns=["df", "Position"])
table = table.withNewColumn("Position(s)", format_positions, columns=["Position"])
table = table.getColumns(["Position(s)", "Deviance", "df", "prob"])
table = table.withNewHeader("prob", "p-value")
table.setColumnFormat("p-value", format_pvalue)
table.setColumnFormat("Deviance", "%.1f")
table.Title = r"Log-linear analysis of A$\rightarrow$G autosomal intergenic mutations. "\
+r"Position(s) are relative to the index position (see Figure \ref{fig:nbr_indexing}). "\
+r"Deviance is from the log-linear model, with df degrees-of-freedom and corresponding "\
+r"p-value obtained from the $\chi^2$ distribution. p-values listed as 0.0 are below "\
+r"the limit of detection."
all_ms_tables.append(format_latex_table(table, justify="rrrl", label="table:a-g"))

### Melanoma mutation summary

In [16]:
fns = !ls ../results/ensembl_snps_79/malignant_melanoma/directions/*/*.txt

summary = make_summary_table(fns)
summary = summary.getColumns([c for c in summary.Header if "4" not in c])
summary.Title = "Summary of neighbourhood contributions to mutations in malignant melanoma. "\
+r"RE$_{max}(\#)$ is the maximum RE for order \# and Pos.$(\#)$ the corresponding position(s). "\
+r"All point mutations had at least "\
+r"one significant test after correcting for 15 tests (see Table \ref{table:c-t}) using the "\
+r"Holm-{\v S}id{\"a}k procedure. Non-significant results are indicated by `-'."
all_ms_tables.append(format_latex_table(summary, justify="crrrrrr", label="table:melanoma:summary"))

In [17]:
with open("../results/ensembl_snps_79/manuscript_tables.tex", "w") as out:
    out.write("\n\n".join(all_ms_tables))

# Supplementary material figures and tables

## Figures

### C->T and A->G figures from exon data

In [18]:
fns = !ls ../results/ensembl_snps_79/A/exon/directions/CtoT/*.pdf
for fn in fns:
    bn = os.path.basename(fn)
    bn = "A_exon_freq_All-chrom_A-GC_All-CtoT_%s" % bn
    ofn = os.path.join("../figures/", bn)
    shutil.copy(fn, ofn)

fns = !ls ../results/ensembl_snps_79/A/exon/directions/AtoG/*.pdf
for fn in fns:
    bn = os.path.basename(fn)
    bn = "A_exon_freq_All-chrom_A-GC_All-AtoG_%s" % bn
    ofn = os.path.join("../figures/", bn)
    shutil.copy(fn, ofn)

### Autosomal intergenic nbr grid

In [19]:
fn = "../results/ensembl_snps_79/A/intergenic/directions/nbr_grid.pdf"
ofn = os.path.join("../figures/", "A_intergenic_nbr_grid.pdf")
shutil.copy(fn, ofn)

### Long flanks for C->T and A->G

In [20]:
fns = !ls ../results/ensembl_snps_79/long_flanks/directions/*/1.pdf
fns = [fn for fn in fns if 'CtoT' in fn or 'AtoG' in fn]
for fn in fns:
    direction = fn.split('/')[-2]
    ofn = "../figures/long_flank_A_intergenic_%s.pdf" % direction
    shutil.copy(fn, ofn)


## Tables

In [21]:
all_supp_tables = []

In [22]:
def DistanceFromMutation(flank_size):
    """returns the number of bases to the mutated position"""
    def call(position):
        # expecting string like "pos14"
        position = int(position[3:])
        dist = position - flank_size
        if dist >= 0:
            # because the position indices don"t consider
            # the mutated position
            dist += 1
        return dist
    return call

def determine_limits(fns, fold=10.0):
    """for each file, identify the neighbourhood size"""
    flank_size = None
    dist_from_mutation = None
    p_dist_from_mutation = None

    rows = []
    for fn in fns:
        direction = format_direction(fn.split("/")[-2])
        table = LoadTable(fn, sep="\t")
        is_sig = SequentialSidak(table.Shape[0])

        flank_size = flank_size or table.Shape[0] / 2
        dist_from_mutation = dist_from_mutation or DistanceFromMutation(flank_size)
        REmax = max(table.getRawData("RE"))
        
        re_table = table.withNewColumn("max_factor", lambda x: x/REmax, columns="RE")
        re_table = re_table.filtered(lambda x: x >= 0.1, columns="max_factor")
        re_table = re_table.withNewColumn("re_dist", dist_from_mutation, columns="Position")
        re_max_dist = max(map(abs, re_table.getRawData("re_dist")))
        
        sig_table = table.filtered(is_sig, columns="prob")
        sig_table = sig_table.withNewColumn("p_dist", dist_from_mutation, columns="Position")
        p_max_dist = max(map(abs, sig_table.getRawData("p_dist")))


        rows.append([direction, REmax, re_max_dist, p_max_dist])
    new = LoadTable(header=["Direction", r"RE$_{max}(1)$", "RE Dist.", "p-val Dist."], rows=rows)
    return new   

### The extent of neighbourhoods affecting mutations

#### Germline intergenic

In [23]:
fns = !ls ../results/ensembl_snps_79/long_flanks/directions/*/*.txt
result = determine_limits(fns)
result.Title = r"The most distant positions from the mutation with RE$(1)\ge10\%$ of RE$_{max}(1)$. "\
+r"RE$(1)$ is the first order RE for the position, and RE$_{max}(1)$ the largest RE from a first order "\
+r"effect for the surveyed positions. RE Dist. is the absolute value of the relative position based "\
+r"on the RE value. p-val Dist. is the corresponding distance based on the p-value. The maximum possible "\
+r"distance is 10. Only point mutations significant after correcting for 20 tests using the "\
+r"Holm-{\v S}id{\"a}k procedure were considered."

all_supp_tables.append(format_latex_table(result.getColumns(["Direction", "RE$_{max}(1)$", "RE Dist.",
                                                             "p-val Dist."]),
                                          justify="crrr", label="suptable:nbr:intergenic-intron:nbrsize"))

### Comparing neghbour effects between autosomal intergenic and intronic

In [24]:
fns = !ls ../results/ensembl_snps_79/A/intergenic_vs_intron/directions/*/summary.txt
fns = [fn for fn in fns if fn.split("/")[-2] in MUTATION_COMPLEMENTS]
summary = make_summary_table(fns)

summary.Title = "Neighbourhood influences on point mutations differ between autosomal "\
+r"intronic and intergenic point mutations. As there was no significant strand asymmetry detected "\
+r"for either sequence class, only + strand effects are shown. "\
+r"Only point mutations with at least "\
+r"one significant test after correcting for 15 tests using the "\
+r"Holm-{\v S}id{\"a}k procedure are shown. Non-significant results are indicated by `-'."
summary = summary.getColumns([c for c in summary.Header if "3" not in c and "4" not in c])
all_supp_tables.append(format_latex_table(summary, justify="cccrr", label="suptable:nbr:intergenic-intron:summary"))

### Comparison of neighbour effects for intergenic Auto and X-chrom

In [25]:
def just_single_positions(table):
    return table.filtered(lambda x: x == 1, columns="motif_length")

In [26]:
fns = !ls ../results/ensembl_snps_79/A_vs_X/intergenic/directions/*/summary.txt
summary = make_summary_table(fns, filter_func=just_single_positions, show_significant=True)
summary = summary.getColumns([c for c in summary.Header if "2" not in c and "3" not in c and "4" not in c])
summary.Title = "Significant differences in neighbourhood effect between intergenic "\
+"autosomal and X-chromosomal point mutations. RE$_{max}(1)$ the largest RE from a first order test and "\
+r"Pos.$(1)$ is the corresponding position. Only mutations significant after correcting for the 15 "\
+r"different tests using the Holm-{\v S}id{\"a}k procedure are shown."
all_supp_tables.append(format_latex_table(summary, justify="ccc", label="suptable:nbr:auto-xchrom"))

AtoC
Position        RE    Deviance    df      prob    motif_length
--------------------------------------------------------------
    pos2    0.0000     13.4929     3    0.0037               1
--------------------------------------------------------------

AtoG
Position        RE    Deviance    df      prob    motif_length
--------------------------------------------------------------
    pos2    0.0000     19.5229     3    0.0002               1
    pos3    0.0000     13.4893     3    0.0037               1
    pos1    0.0000      9.6424     3    0.0219               1
--------------------------------------------------------------

CtoG
Position        RE    Deviance    df      prob    motif_length
--------------------------------------------------------------
    pos2    0.0000     12.0749     3    0.0071               1
--------------------------------------------------------------

CtoT
Position        RE    Deviance    df      prob    motif_length
--------------------------------

### Comparison of spectra for intergenic and intronic for Autosomal

In [27]:
from itertools import combinations

is_sig = SequentialSidak(4)
sig_comp_tables = {}

comb = "intergenic_vs_intron"
path = "../results/ensembl_snps_79/A/%s/spectra_summary.txt" % comb
table = LoadTable(path, sep="\t")
table = table.filtered(is_sig, columns="prob")
table = table.withNewColumn("Direction", lambda x: format_direction(x), columns="direction")
table = table.withNewColumn("Group", lambda x: list(set(["intergenic","intron"]) & set(x.split("/")))[0], columns="group")
table = table.filtered(lambda x: x == "intergenic", columns="Group")
table = table.getColumns(["Direction", "ret"])
table = table.withNewHeader(["ret"], ["RET"])
table.Title = "Significant differences in mutation spectra between autosomal intergenic and "\
+r"intronic point mutations. Separate log-linear models were used for each starting base ($X$ in X$\rightarrow$Y). "\
+r"RET is the RE term for that row mutation direction. Only RET from the intergenic group are shown. A positive (negative) "\
+r"RET indicates a excess (deficit) of that mutation in the intergenic group. All tests returned p-values that were below "\
+r"the limit of detection and thus were statistically significant after correcting for 4 tests using "\
+r"the Holm-{\v S}id{\"a}k procedure."
all_supp_tables.append(format_latex_table(table, justify="ccrr", label="suptable:spectra:intergenic-intron"))

### Comparison of spectra for intergenic, between Auto and X-chrom

In [28]:
path = "../results/ensembl_snps_79/A_vs_X/intergenic/spectra_summary.txt"
is_sig = SequentialSidak(4)
table = LoadTable(path, sep="\t")
table = table.filtered(is_sig, columns="prob")
table = table.withNewColumn("Chrom. Class",
                            lambda x: list(set(["A", "X"]) & set(x.split("/")))[0], columns="group")
table = table.withNewColumn("Direction", lambda x: format_direction(x), columns="direction")
table = table.filtered(lambda x: x == "A", columns="Chrom. Class")
table = table.getColumns(["Direction", "ret"])
table = table.withNewHeader(["ret"], ["RET"])
table.Title = "Significant differences in spectra between autosomal and X-chromosomal intergenic "\
+r"point mutations. Separate log-linear models were used for each starting base ($X$ in X$\rightarrow$Y). "\
+r"RET is the RE term for that row mutation direction. p-value is from the corresponding hypothesis test. "\
+r"Only RET from the autosomal group are shown. A positive (negative) "\
+r"RET indicates a excess (deficit) of that mutation in autosomes. All tests returned p-values that were $\le 4.7e^{-9}$ "\
+r"and thus were statistically significant after correcting for 4 tests using the Holm-{\v S}id{\"a}k procedure."
all_supp_tables.append(format_latex_table(table, justify="ccrr", label="suptable:spectra:autosome-xchrom:intergenic"))

### Comparison of spectra for intronic, between Auto and X-chrom

In [29]:
path = "../results/ensembl_snps_79/A_vs_X/intron/spectra_summary.txt"
is_sig = SequentialSidak(4)
table = LoadTable(path, sep="\t")
table = table.filtered(is_sig, columns="prob")
table = table.withNewColumn("Chrom. Class",
                            lambda x: list(set(["A", "X"]) & set(x.split("/")))[0], columns="group")
table = table.withNewColumn("Direction", lambda x: format_direction(x), columns="direction")
table = table.filtered(lambda x: x == "A", columns="Chrom. Class")
table = table.getColumns(["Direction", "ret"])
table = table.withNewHeader(["ret"], ["RET"])
table.Title = "Significant differences in spectra between autosomal and X-chromosomal intronic "\
+r"point mutations. Separate log-linear models were used for each starting base ($X$ in X$\rightarrow$Y). "\
+r"RET is the RE term for that row mutation direction. p-value is from the corresponding hypothesis test. "\
+r"Only RET from the autosomal group are shown. A positive (negative) "\
+r"RET indicates a excess (deficit) of that mutation in autosomes. All tests returned p-values that were $\le 8.6e^{-5}$ "\
+r"and thus were statistically significant after correcting for 4 tests using the Holm-{\v S}id{\"a}k procedure."

all_supp_tables.append(format_latex_table(table, justify="ccrr", label="suptable:spectra:autosome-xchrom:intron"))

### Strand symmetry neighbour for malignant melanoma

In [30]:
fns = !ls ../results/ensembl_snps_79/malignant_melanoma/directions_strand_symmetric/*/*.txt
summary = make_summary_table(fns)
summary = summary.getColumns([c for c in summary.Header if "4" not in c])
summary.Title = "Test of strand symmetric neighbourhood influences on malignant melanoma point mutations. "\
+r"RE$_{max}(\#)$ is the maximum RE for order "\
+r"\# and Pos.$(\#)$ the corresponding position(s). Only effects significant after "\
+r"correcting for the 15 different tests using the Holm-{\v S}id{\"a}k procedure are shown. "\
+r"Non-significant results are indicated by `-'."
all_supp_tables.append(format_latex_table(summary, justify="cccrrrr", label="suptable:nbr:melanoma:summary"))

### Strand symmetry spectra for malignant melanoma

In [31]:
is_sig = SequentialSidak(4)
table = LoadTable("../results/ensembl_snps_79/malignant_melanoma/combined_strand_symmetric/spectra_summary.txt", sep="\t")
table = table.filtered(is_sig, columns="prob")
if table.Shape[0] == 0:
    print "No significant differences in spectra between %s" % seq_class
print
table = table.withNewColumn("Direction", lambda x: format_direction(x), columns="direction")
table = table.filtered(lambda x: x == "+", columns="strand")
table = table.getColumns(["Direction", "ret", "prob"])
table = table.withNewHeader(["ret", "prob"], ["RET", "p-value"])
table.setColumnFormat("p-value", format_pvalue)
table.Title = "Differences in spectra between strands for malignant melanoma point mutations. "\
+r"Separate log-linear models were used for the + strand starting bases A and C. "\
+r"RET is the RE term for that row mutation direction. "\
+r"Only RET from the + strand are shown. A positive (negative) "\
+r"RET indicates a excess (deficit) of that mutation on the + strand. p-value is from the corresponding hypothesis test. "\
+r"Only mutations from C were significant after correcting for 2 tests using the Holm-{\v S}id{\"a}k procedure."
all_supp_tables.append(format_latex_table(table.getColumns(["Direction", "RET", "p-value"]), justify="ccccrrr",
                         label="suptable:spectra:melanoma:symmetry"))

### Intronic autosomal mutation summary

In [32]:
fns = !ls ../results/ensembl_snps_79/A/intron/directions/*/*.txt
fns = [fn for fn in fns if fn.split("/")[-2] in MUTATION_COMPLEMENTS]
summary = make_summary_table(fns)
summary = summary.getColumns([c for c in summary.Header if "4" not in c])
summary.Title = "Neighbourhood influences on point mutations within autosomal introns. "\
+r"RE$_{max}(1)$ the largest RE from a first order test and "\
+r"Pos.$(1)$ is the corresponding position. Only mutations significant after correcting for the 15 "\
+r"different tests using the Holm-{\v S}id{\"a}k procedure are shown."
all_supp_tables.append(format_latex_table(summary, justify="crrrrrr", label="suptable:nbr:auto-intron:summary"))

### Neighbour influence differences between germline exon and melanoma

In [33]:
fns = !ls ../results/ensembl_snps_79/Aexon_vs_Melanoma/directions/*/summary.txt
summary = make_summary_table(fns, show_significant=False)
summary.Title = "Significant differences in the influence of neighbours on "\
+r"exonic point mutations between germline and malignant melanoma. "\
+r"RE$_{max}(1)$ the largest RE from a first order test and "\
+r"Pos.$(1)$ is the corresponding position. Only mutations significant after correcting for the 15 "\
+r"different tests using the Holm-{\v S}id{\"a}k procedure are shown. Non-significant results are indicated by `-'."
summary = summary.getColumns([c for c in summary.Header if "4" not in c])
all_supp_tables.append(format_latex_table(summary, justify="crrrrrr", label="suptable:nbr:exon-germline-melanoma:summary"))

### Spectra differences between germline exon and melanoma

In [34]:
table = LoadTable("../results/ensembl_snps_79/Aexon_vs_Melanoma/spectra_summary.txt", sep="\t")
is_sig = SequentialSidak(4)
table = table.filtered(is_sig, columns="prob")
if table.Shape[0] == 0:
    print "No significant differences in spectra between germline exon and melanoma"
print
table = table.withNewColumn("Direction", lambda x: format_direction(x), columns="direction")
table = table.withNewColumn("Group",
                                lambda x: "melanoma" if "malignant_melanoma" in x else "germline", columns="group")
table = table.filtered(lambda x: x == "melanoma", columns="Group")
table = table.getColumns(["Direction", "ret"])
table = table.withNewHeader(["ret"], ["RET"])
table = table.getColumns(["Direction", "RET"])
table.Title = "Significant differences in spectra between germline exon and malignant melanoma "\
+r"point mutations. Separate log-linear models were used for each starting base ($X$ in X$\rightarrow$Y). "\
+r"RET is the RE term for that row mutation direction. Only RET from the melanoma group are shown. A positive (negative) "\
+r"RET indicates a excess (deficit) of that mutation in the melanoma group. All tests returned p-values that were below "\
+r"the limit of detection and thus were statistically significant after correcting for 4 tests using "\
+r"the Holm-{\v S}id{\"a}k procedure."
all_supp_tables.append(format_latex_table(table, justify="ccrrr", label="suptable:spectra:exon-germline-melanoma"))

In [35]:
with open("../results/ensembl_snps_79/supp_materials_tables.tex", "w") as out:
    out.write("\n\n".join(all_supp_tables))